In [ ]:
!ls

drive  sample_data


In [ ]:
cd drive/MyDrive/DeepLearningCourse/final project

/content/drive/MyDrive/DeepLearningCourse/final project


In [ ]:
import os
import sys
import random
import argparse
import json
import nltk
import numpy as np
from tqdm import tqdm
from six.moves.urllib.request import urlretrieve
import unicodedata
import json
random.seed(42)
np.random.seed(42)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.2 MB/s eta 0:00:00


In [ ]:
pip install sentencepiece

In [ ]:
import collections
def compute_f1(a_gold, a_pred):
    common = collections.Counter(a_gold) & collections.Counter(a_pred)
    num_same = sum(common.values())
    if len(a_gold) == 0 or len(a_pred) == 0:
      # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
      return int(a_gold == a_pred)
    if num_same == 0:
      return 0
    precision = 1.0 * num_same / len(a_pred)
    recall = 1.0 * num_same / len(a_gold)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def compute_exact(a_gold, a_pred):
    return int(a_gold == a_pred)

###data preprocessing


clean data, save to json

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def data_from_json(filename):
    """Loads JSON data from filename and returns"""
    with open(filename) as data_file:
        data = json.load(data_file)
    return data

In [ ]:
def total_exs(dataset):
    """
    Returns the total number of (context, question, answer) triples,
    given the data read from the SQuAD json file.
    """
    total = 0
    for article in dataset['data']:
        for para in article['paragraphs']:
            total += len(para['qas'])
    return total

In [ ]:

def tokenize(sequence):
    tokens = [token.replace("``", '"').replace("''", '"').lower() for token in nltk.word_tokenize(sequence)]
    return tokens

def get_char_word_loc_mapping(context, context_tokens):
    """
    Return a mapping that maps from character locations to the corresponding token locations.
    If we're unable to complete the mapping e.g. because of special characters, we return None.
    Inputs:
      context: string (unicode)
      context_tokens: list of strings (unicode)
    Returns:
      mapping: dictionary from ints (character locations) to (token, token_idx) pairs
        Only ints corresponding to non-space character locations are in the keys
        e.g. if context = "hello world" and context_tokens = ["hello", "world"] then
        0,1,2,3,4 are mapped to ("hello", 0) and 6,7,8,9,10 are mapped to ("world", 1)
    """
    acc = '' # accumulator
    current_token_idx = 0 # current word loc
    mapping = dict()

    for char_idx, char in enumerate(context): # step through original characters
        if char != u' ' and char != u'\n': # if it's not a space:
            acc += char # add to accumulator
            context_token = unicodedata.normalize('NFKD', context_tokens[current_token_idx]) # current word token
            if acc == context_token: # if the accumulator now matches the current word token
                syn_start = char_idx - len(acc) + 1 # char loc of the start of this word
                for char_loc in range(syn_start, char_idx+1):
                    mapping[char_loc] = (acc, current_token_idx) # add to mapping
                acc = '' # reset accumulator
                current_token_idx += 1

    if current_token_idx != len(context_tokens):
        return None
    else:
        return mapping

def preprocess_and_write(dataset, tier, out_dir):
    """Reads the dataset, extracts context, question, answer, tokenizes them
    and calculates answer span in terms of token indices.
    Note: due to tokenization issues, and the fact that the original answer
    spans are given in terms of characters, some examples are discarded because
    we cannot get a clean span in terms of tokens.
    This function produces the {train/dev}_{context/question/answer/span} files.
    Inputs:
      dataset: read from JSON
      tier: string ("train" or "dev")
      out_dir: directory to write the preprocessed files
    Returns:
      the number of (context, question, answer) triples written to file by the dataset.
    """

    num_exs = 0 # number of examples written to file
    num_mappingprob, num_tokenprob, num_spanalignprob = 0, 0, 0
    examples = []

    for articles_id in tqdm(range(len(dataset['data'])), desc="Preprocessing {}".format(tier)):

        article_paragraphs = dataset['data'][articles_id]['paragraphs']
        for pid in range(len(article_paragraphs)):

            context = unicodedata.normalize('NFKD', article_paragraphs[pid]['context']) # string

            # The following replacements are suggested in the paper
            # BidAF (Seo et al., 2016)
            context = context.replace("''", '" ')
            context = context.replace("``", '" ')

            context_tokens = tokenize(context) # list of strings (lowercase)
            context = context.lower()

            qas = article_paragraphs[pid]['qas'] # list of questions

            charloc2wordloc = get_char_word_loc_mapping(context, context_tokens) # charloc2wordloc maps the character location (int) of a context token to a pair giving (word (string), word loc (int)) of that token

            if charloc2wordloc is None: # there was a problem
                num_mappingprob += len(qas)
                continue # skip this context example

            # for each question, process the question and answer and write to file
            for qn in qas:

                # read the question text and tokenize
                question = unicodedata.normalize('NFKD', qn['question']) # string
                question_tokens = tokenize(question) # list of strings

                # of the three answers, just take the first
                if len(qn['answers'])!=0:
                    ans_text = unicodedata.normalize('NFKD', qn['answers'][0]['text']).lower() # get the answer text
                    ans_start_charloc = qn['answers'][0]['answer_start'] # answer start loc (character count)
                    ans_end_charloc = ans_start_charloc + len(ans_text) # answer end loc (character count) (exclusive)

                    # Check that the provided character spans match the provided answer text
                    if context[ans_start_charloc:ans_end_charloc] != ans_text:
                      # Sometimes this is misaligned, mostly because "narrow builds" of Python 2 interpret certain Unicode characters to have length 2 https://stackoverflow.com/questions/29109944/python-returns-length-of-2-for-single-unicode-character-string
                      # We should upgrade to Python 3 next year!
                      num_spanalignprob += 1
                      continue

                    # get word locs for answer start and end (inclusive)
                    ans_start_wordloc = charloc2wordloc[ans_start_charloc][1] # answer start word loc
                    ans_end_wordloc = charloc2wordloc[ans_end_charloc-1][1] # answer end word loc
                    assert ans_start_wordloc <= ans_end_wordloc

                    # Check retrieved answer tokens match the provided answer text.
                    # Sometimes they won't match, e.g. if the context contains the phrase "fifth-generation"
                    # and the answer character span is around "generation",
                    # but the tokenizer regards "fifth-generation" as a single token.
                    # Then ans_tokens has "fifth-generation" but the ans_text is "generation", which doesn't match.
                    ans_tokens = context_tokens[ans_start_wordloc:ans_end_wordloc+1]
                    if "".join(ans_tokens) != "".join(ans_text.split()):
                        num_tokenprob += 1
                        continue # skip this question/answer pair
                else:
                    ans_text = 'not in the paragraph'
                    ans_start_charloc = -1
                    ans_end_charloc = -1

                # examples.append((' '.join(context_tokens), ' '.join(question_tokens), ' '.join(ans_tokens), ' '.join([str(ans_start_wordloc), str(ans_end_wordloc)])))
                examples.append((context,question,ans_text,ans_start_charloc,ans_end_charloc))
                # print(ans_text,ans_start_charloc,ans_end_charloc)

                num_exs += 1

    print("\nNumber of (context, question, answer) triples discarded due to char -> token mapping problems: ", num_mappingprob)
    print("Number of (context, question, answer) triples discarded because character-based answer span is unaligned with tokenization: ", num_tokenprob)
    print("Number of (context, question, answer) triples discarded due character span alignment problems (usually Unicode problems): ", num_spanalignprob)
    print("Processed %i examples of total %i\n" % (num_exs, num_exs + num_mappingprob + num_tokenprob + num_spanalignprob))

    with open(out_dir+"/"+tier+".json", 'w', encoding='utf-8') as json_file:
        # shuffle examples
        random.shuffle(examples)

        dictionary = {"data": examples}
        # Convert dictionary to JSON
        json_data = json.dumps(dictionary)
        json_file.write(json_data)


In [ ]:
train_filename = "train-v2.0.json"
dev_filename = "dev-v2.0.json"

train_data = data_from_json(train_filename)
dev_data = data_from_json(dev_filename)
print( "Train data has %i examples total" % total_exs(train_data))
print( "Dev data has %i examples total" % total_exs(dev_data))
preprocess_and_write(train_data, 'train', "preprocessed_data")
preprocess_and_write(dev_data, 'val', "preprocessed_data")

Train data has 130319 examples total
Dev data has 11873 examples total


Preprocessing train: 100%|██████████| 442/442 [00:34<00:00, 12.86it/s]



Number of (context, question, answer) triples discarded due to char -> token mapping problems:  0
Number of (context, question, answer) triples discarded because character-based answer span is unaligned with tokenization:  1071
Number of (context, question, answer) triples discarded due character span alignment problems (usually Unicode problems):  6089
Processed 123159 examples of total 130319



Preprocessing val: 100%|██████████| 35/35 [00:02<00:00, 12.49it/s]



Number of (context, question, answer) triples discarded due to char -> token mapping problems:  0
Number of (context, question, answer) triples discarded because character-based answer span is unaligned with tokenization:  67
Number of (context, question, answer) triples discarded due character span alignment problems (usually Unicode problems):  346
Processed 11460 examples of total 11873



### Models

In [ ]:
import transformers
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, AdamW
from transformers import ElectraForQuestionAnswering, ElectraTokenizer
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn

prepare model input data structure

In [ ]:
train_data = json.load(open('preprocessed_data/train.json'))
val_data = json.load(open('preprocessed_data/val.json'))

In [ ]:
train_data = train_data['data'][:10000]
val_data = val_data['data'][:1000]

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_seq_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __getitem__(self, idx):
        context, question, answer, start_position, end_position = self.data[idx]

        encoding = self.tokenizer.encode_plus(
            question,
            context,
            add_special_tokens=True,
            max_length=self.max_seq_length,
            truncation="only_second",  # Truncate context, not the question
            padding="max_length",
            return_tensors="pt"
        )

        input_ids = encoding["input_ids"].squeeze()
        attention_mask = encoding["attention_mask"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "start_positions": start_position,
            "end_positions": end_position
        }

    def __len__(self):
        return len(self.data)


In [ ]:
for a,b,c,d,e in train_data[:10]:
  print(c,d,e)

bourgeois stores 506 522
not in the paragraph -1 -1
not in the paragraph -1 -1
not in the paragraph -1 -1
radio silence was observed until the bombs fell 326 373
understand the meaning of the words they are singing. 619 672
jiangnan examination hall 361 386
fourth 453 459
not in the paragraph -1 -1
not in the paragraph -1 -1


### base models

In [ ]:
# model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")
# tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

model = ElectraForQuestionAnswering.from_pretrained("google/electra-small-discriminator", output_hidden_states=True)
tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['qa_outputs.bias', 'qa_outputs.

In [ ]:
batch_size = 32
train_dataset = CustomDataset(train_data, tokenizer, max_seq_length=384)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = CustomDataset(val_data, tokenizer, max_seq_length=384)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
torch.cuda.empty_cache()
model.to(device)

ElectraForQuestionAnswering(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(

In [ ]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
        self.loss_fct = nn.CrossEntropyLoss()

    def compute_loss(self, start_logits, start_logits_pos, start_positions, end_logits, end_logits_pos, end_positions):
        # print(start_logits, start_positions)
        start_loss = self.helper_compute_loss(start_logits, start_logits_pos, start_positions)
        end_loss = self.helper_compute_loss(end_logits, end_logits_pos, end_positions)
        total_loss = (start_loss + end_loss) / 2.0
        return total_loss

    def helper_compute_loss(self, logits, logits_pos, positions):
        mask = (positions != -1) | (logits >= 0.5)

        non_valid_logits = logits[~mask]
        non_valid_positions = torch.ones_like(positions[~mask])

        valid_logits = logits[mask]
        valid_positions = positions[mask]

        loss = torch.tensor(0.0, dtype=torch.float32, device=logits.device)
        loss_a = torch.tensor(0.0, dtype=torch.float32, device=logits.device)
        loss_b = torch.tensor(0.0, dtype=torch.float32, device=logits.device)

        if valid_logits.size(0) > 0:
            # loss_a += self.loss_fct(valid_logits_pos, valid_positions)/ valid_positions.size(0)
            loss_a += self.loss_fct(valid_logits, valid_positions)/ valid_positions.size(0)
        if non_valid_logits.size(0) > 0:
            # loss_b += self.loss_fct(non_valid_logits_pos, non_valid_positions)/ non_valid_positions.size(0)
            loss_b += self.loss_fct(non_valid_logits, non_valid_positions)/ non_valid_positions.size(0)
        loss += loss_a
        loss += loss_b
        return loss

In [ ]:
num_epochs = 1
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_function = CustomLoss()

In [ ]:
torch.autograd.set_detect_anomaly(True)


In [ ]:
model.train()
plot_f1 = []
plot_em = []
plot_loss = []
for epoch in range(num_epochs):
    f1_scores = []
    em_scores = []
    losses = []
    i=0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        start_positions = batch["start_positions"].to(device)
        end_positions = batch["end_positions"].to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # start_positions=start_positions,
            # end_positions=end_positions
        )
        print(outputs.start_logits.shape)

        start_logits, start_logits_pos = torch.max(outputs.start_logits, dim=1, keepdim=True)
        end_logits, end_logits_pos = torch.max(outputs.end_logits, dim=1, keepdim=True)
        start_logits_pos = start_logits_pos.float().to(device)
        end_logits_pos = end_logits_pos.float().to(device)
        start_logits_pos.requires_grad = True
        end_logits_pos.requires_grad = True

        loss = loss_function.compute_loss(
            start_logits.squeeze().float(), start_logits_pos.squeeze().float(),
            start_positions.float(), end_logits.squeeze().float(),
            end_logits_pos.squeeze().float(), end_positions.float()
            )
        losses.extend([loss.item()])

        context_ids = input_ids.detach().cpu().numpy()


        # Calculate F1 score and EM
        for (start, end), (pred_start_prob, pred_end_prob), (pred_start, pred_end), context in zip(zip(start_positions, end_positions), zip(start_logits, end_logits), zip(start_logits_pos, end_logits_pos), context_ids):
            start, end, pred_start, pred_end = int(start), int(end), int(pred_start), int(pred_end)
            # true_answer
            if start == -1 and end == -1:
                true_answer = "not in the paragraph"
            else:
                true_answer = tokenizer.decode(context[start:end+1])

            # pred_answer
            threshold = 0.5
            if pred_start_prob < threshold or pred_end_prob < threshold:
                pred_answer = "not in the paragraph"
            else:
                pred_answer = tokenizer.decode(context[pred_start:pred_end+1])
            f1_scores.append(compute_f1(true_answer, pred_answer))
            em_scores.append(compute_exact(true_answer, pred_answer))

        f = (batch_size*i)/10000
        if i%20==0:
            print(f'finished {f*100:.2f}%, loss:{loss.item():.2f}')
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
        i+=1
    i=0

    model.eval()
    val_f1_scores = []
    val_em_scores = []
    val_losses = []

    with torch.no_grad():
        for val_batch in val_loader:
            input_ids = val_batch["input_ids"].to(device)
            attention_mask = val_batch["attention_mask"].to(device)
            start_positions = val_batch["start_positions"].to(device)
            end_positions = val_batch["end_positions"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                start_positions=start_positions,
                end_positions=end_positions
            )

            start_logits, start_logits_pos = torch.max(outputs.start_logits, dim=1, keepdim=True)
            end_logits, end_logits_pos = torch.max(outputs.end_logits, dim=1, keepdim=True)

            val_loss = loss_function.compute_loss(
                start_logits.squeeze().float(), start_logits_pos.squeeze().float(),
                start_positions.float(), end_logits.squeeze().float(),
                end_logits_pos.squeeze().float(), end_positions.float()
            )
            val_losses.extend([val_loss.item()])

            start_logits_pos = start_logits_pos.detach().cpu().numpy()
            end_logits_pos = end_logits_pos.detach().cpu().numpy()
            context_ids = input_ids.detach().cpu().numpy()
            start_positions = start_positions.detach().cpu().numpy()
            end_positions = end_positions.detach().cpu().numpy()

            # Calculate F1 score and EM
            for (start, end), (pred_start_prob, pred_end_prob), (pred_start, pred_end), context in zip(zip(start_positions, end_positions), zip(start_logits, end_logits), zip(start_logits_pos, end_logits_pos), context_ids):
                start, end, pred_start, pred_end = int(start), int(end), int(pred_start), int(pred_end)
                # true_answer
                if start == -1 and end == -1:
                    true_answer = "not in the paragraph"
                else:
                    true_answer = tokenizer.decode(context[start:end+1])

                # pred_answer
                threshold = 0.5
                if pred_start_prob < threshold or pred_end_prob < threshold:
                    pred_answer = "not in the paragraph"
                else:
                    pred_answer = tokenizer.decode(context[pred_start:pred_end+1])
                val_f1_scores.append(compute_f1(true_answer, pred_answer))
                val_em_scores.append(compute_exact(true_answer, pred_answer))


    # Perform evaluation
    avg_f1_score = np.mean(f1_scores)
    avg_em_score = np.mean(em_scores)
    avg_loss = np.mean(losses)

    avg_val_f1_score = np.mean(val_f1_scores)
    avg_val_em_score = np.mean(val_em_scores)
    avg_val_loss = np.mean(val_losses)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - F1 Score: {avg_f1_score:.4f}, EM Score: {avg_em_score:.4f}, Loss: {avg_loss:.4f}")
    print(f"Validation - F1 Score: {avg_val_f1_score:.4f}, EM Score: {avg_val_em_score:.4f}, Loss: {avg_val_loss:.4f}")
    plot_f1.append(avg_f1_score)
    plot_em.append(avg_em_score)
    plot_loss.append(avg_loss)

    model.train()


torch.Size([2, 384])
finished 0.00%, loss:176.45


KeyboardInterrupt: ignored

In [ ]:
model.save_pretrained("DistilAlbertModel")
# model.save_pretrained("ElectraModel")

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
num_params = count_parameters(model)
print(f"Number of trainable parameters: {num_params}")

### my model

In [ ]:
import torch.nn.functional as F
import torch.nn as nn

In [ ]:
import torch
import torch.nn as nn
from transformers import ElectraModel, ElectraConfig

class ElectraBiDirectionalAttention(nn.Module):
    def __init__(self, electra_model, hidden_size, num_labels, num_heads=8):
        super(ElectraBiDirectionalAttention, self).__init__()
        self.electra = electra_model
        self.attention = nn.MultiheadAttention(hidden_size, num_heads)
        self.dropout = nn.Dropout(0.2)
        self.attention_linear = nn.Linear(hidden_size, hidden_size * 2)
        self.start_classifier = nn.Linear(hidden_size * 2, 1)
        self.end_classifier = nn.Linear(hidden_size * 2, 1)

    def forward(self, input_ids, attention_mask, start_positions, end_positions):
        outputs = self.electra(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        sequence_output = self.dropout(sequence_output)
        print(sequence_output.shape)  # torch.Size([2, 384, 768])

        # Transpose sequence_output to have shape (seq_len, batch_size, hidden_size)
        sequence_output = sequence_output.transpose(0, 1)

        # Apply multihead attention
        attention_output, attention_output_weights = self.attention(sequence_output, sequence_output, sequence_output)
        attention_output = attention_output.transpose(0, 1)
        print(attention_output.shape)  # torch.Size([2, 384, 768])

        # Apply linear layer to reshape attention_output
        attention_output = self.attention_linear(attention_output)

        start_logits = self.start_classifier(attention_output).squeeze(-1)
        end_logits = self.end_classifier(attention_output).squeeze(-1)

        output = {
            "start_logits": start_logits,
            "end_logits": end_logits
        }

        return output

num_heads = 8


In [ ]:
import torch.nn as nn

class ElectraBiLSTMAttention(nn.Module):
    def __init__(self, electra_model, hidden_size, num_labels):
        super(ElectraBiLSTMAttention, self).__init__()
        self.electra = electra_model
        self.bilstm = nn.LSTM(hidden_size, hidden_size, batch_first=True, bidirectional=True)
        self.attention = nn.Linear(hidden_size * 2, 1)
        self.start_classifier = nn.Linear(hidden_size * 2, 384)
        self.end_classifier = nn.Linear(hidden_size * 2, 384)

    def forward(self, input_ids, attention_mask):
        outputs = self.electra(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        lstm_output, _ = self.bilstm(sequence_output)

        attention_scores = self.attention(lstm_output).squeeze(-1)
        attention_weights = torch.softmax(attention_scores, dim=1).unsqueeze(-1)
        attention_output = (lstm_output * attention_weights).sum(dim=1)

        start_logits = self.start_classifier(attention_output)
        end_logits = self.end_classifier(attention_output)

        return start_logits, end_logits

In [ ]:
from transformers import ElectraModel

electra_model = ElectraModel.from_pretrained("google/electra-small-discriminator", output_hidden_states = True)
hidden_size = electra_model.config.hidden_size
num_labels = 2  # start and end positions
# model = ElectraBiDirectionalAttention(electra_model, hidden_size, num_labels)
model = ElectraBiLSTMAttention(electra_model, hidden_size, num_labels)
tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
batch_size = 32
train_dataset = CustomDataset(train_data, tokenizer, max_seq_length=384)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = CustomDataset(val_data, tokenizer, max_seq_length=384)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
model.to(device)

ElectraBiLSTMAttention(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_fe

In [ ]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
        self.loss_fct = nn.CrossEntropyLoss()

    def compute_loss(self, start_logits, start_positions, end_logits, end_positions):
        # print(start_logits, start_positions)
        start_loss = self.helper_compute_loss(start_logits, start_positions)
        end_loss = self.helper_compute_loss(end_logits, end_positions)
        total_loss = (start_loss + end_loss) / 2.0
        return total_loss

    def helper_compute_loss(self, logits, positions):
        mask = (positions != -1) | (logits >= 0.5)

        non_valid_logits = logits[~mask]
        non_valid_positions = torch.ones_like(positions[~mask])

        valid_logits = logits[mask]
        valid_positions = positions[mask]

        loss = torch.tensor(0.0, dtype=torch.float32, device=logits.device)
        loss_a = torch.tensor(0.0, dtype=torch.float32, device=logits.device)
        loss_b = torch.tensor(0.0, dtype=torch.float32, device=logits.device)

        if valid_logits.size(0) > 0:
            loss_a += self.loss_fct(valid_logits, valid_positions)/ valid_positions.size(0)
        if non_valid_logits.size(0) > 0:
            loss_b += self.loss_fct(non_valid_logits, non_valid_positions)/ non_valid_positions.size(0)
        loss += loss_a
        loss += loss_b
        return loss

In [ ]:
num_epochs = 10
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_function = CustomLoss()

In [ ]:
plot_f1 = []
plot_em = []
plot_loss = []
model.train()
for epoch in range(num_epochs):
    f1_scores = []
    em_scores = []
    losses = []

    i = 0
    for batch in train_loader:
        optimizer.zero_grad()

        # Prepare batch data
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        # every tensor involved in the backward propagation need to set requires_grad = True, some are set True automatically
        start_positions = batch['start_positions'].float().to(device).requires_grad_(True)
        end_positions = batch['end_positions'].float().to(device).requires_grad_(True)

        # Forward pass
        start_logits, end_logits = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # start_positions = start_positions,
            # end_positions = end_positions
        )

        start_logits, start_logits_pos = torch.max(start_logits, dim=1, keepdim=True)
        end_logits, end_logits_pos = torch.max(end_logits, dim=1, keepdim=True)

        # Calculate loss
        loss = loss_function.compute_loss(
            start_logits.squeeze().float(),  start_positions.float(),
            end_logits.squeeze().float(),  end_positions.float()
            )
        losses.extend([loss.item()])
        context_ids = input_ids.detach().cpu().numpy()

        start_logits = start_logits.detach().cpu().numpy()
        end_logits = end_logits.detach().cpu().numpy()
        start_positions = start_positions.detach().cpu().numpy()
        end_positions = end_positions.detach().cpu().numpy()

        # Calculate F1 score and EM
        for (start, end), (pred_start_prob, pred_end_prob), (pred_start, pred_end), context in zip(zip(start_positions, end_positions), zip(start_logits, end_logits), zip(start_logits_pos, end_logits_pos), context_ids):
            start, end, pred_start, pred_end = int(start), int(end), int(pred_start), int(pred_end)
            # true_answer
            if start == -1 and end == -1:
                true_answer = "not in the paragraph"
            else:
                true_answer = tokenizer.decode(context[start:end+1])

            # pred_answer
            threshold = 0.5
            if pred_start_prob < threshold or pred_end_prob < threshold:
                pred_answer = "not in the paragraph"
            else:
                pred_answer = tokenizer.decode(context[pred_start:pred_end+1])
            f1_scores.append(compute_f1(true_answer, pred_answer))
            em_scores.append(compute_exact(true_answer, pred_answer))

        f = (batch_size*i)/10000
        if i%20==0:
            print(f'finished {f*100:.2f}%, loss:{loss.item():.2f}')
        i+=1
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()

    i=0

    model.eval()
    val_f1_scores = []
    val_em_scores = []
    val_losses = []

    with torch.no_grad():
        for val_batch in val_loader:
            input_ids = val_batch["input_ids"].to(device)
            attention_mask = val_batch["attention_mask"].to(device)
            start_positions = val_batch["start_positions"].to(device)
            end_positions = val_batch["end_positions"].to(device)

            start_logits, end_logits = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                # start_positions=start_positions,
                # end_positions=end_positions
            )
            # logits = outputs.transpose(0, 1)
            # start_logits = logits[0]
            # end_logits = logits[1]

            start_logits, start_logits_pos = torch.max(start_logits, dim=1, keepdim=True)
            end_logits, end_logits_pos = torch.max(end_logits, dim=1, keepdim=True)

            val_loss = loss_function.compute_loss(
                start_logits.squeeze().float(),  start_positions.float(),
                end_logits.squeeze().float(),  end_positions.float()
            )
            val_losses.extend([val_loss.item()])

            start_logits_pos = start_logits_pos.detach().cpu().numpy()
            end_logits_pos = end_logits_pos.detach().cpu().numpy()
            context_ids = input_ids.detach().cpu().numpy()
            start_positions = start_positions.detach().cpu().numpy()
            end_positions = end_positions.detach().cpu().numpy()

            # Calculate F1 score and EM
            for (start, end), (pred_start_prob, pred_end_prob), (pred_start, pred_end), context in zip(zip(start_positions, end_positions), zip(start_logits, end_logits), zip(start_logits_pos, end_logits_pos), context_ids):
                start, end, pred_start, pred_end = int(start), int(end), int(pred_start), int(pred_end)
                # true_answer
                if start == -1 and end == -1:
                    true_answer = "not in the paragraph"
                else:
                    true_answer = tokenizer.decode(context[start:end+1])

                # pred_answer
                threshold = 0.5
                if pred_start_prob < threshold or pred_end_prob < threshold:
                    pred_answer = "not in the paragraph"
                else:
                    pred_answer = tokenizer.decode(context[pred_start:pred_end+1])
                val_f1_scores.append(compute_f1(true_answer, pred_answer))
                val_em_scores.append(compute_exact(true_answer, pred_answer))


    # Perform evaluation
    avg_f1_score = np.mean(f1_scores)
    avg_em_score = np.mean(em_scores)
    avg_loss = np.mean(losses)

    avg_val_f1_score = np.mean(val_f1_scores)
    avg_val_em_score = np.mean(val_em_scores)
    avg_val_loss = np.mean(val_losses)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train - F1 Score: {avg_f1_score:.4f}, EM Score: {avg_em_score:.4f}, Loss: {avg_loss:.4f}")
    print(f"Validation - F1 Score: {avg_val_f1_score:.4f}, EM Score: {avg_val_em_score:.4f}, Loss: {avg_val_loss:.4f}")
    plot_f1.append(avg_f1_score)
    plot_em.append(avg_em_score)
    plot_loss.append(avg_loss)

    model.train()

